In [ ]:
import itertools
import random
import os
import sys
sys.path.insert(0,'..')
import psutil

from btgym import BTgymEnv
from btgym.monitor import BTgymMonitor

In [ ]:
env = BTgymEnv(filename='../examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
               state_shape=(30,4),
               skip_frame=10,
               start_cash=100,
               drawdown_call=90,
               render_enabled=True,
               render_ylabel='Price Lines',
               render_size_episode=(12,8),
               render_size_human=(8, 3.5),
               render_size_agent=(10, 3.5),
               render_dpi=75,
               verbose=0,
              )

In [ ]:
# Adding up monitors:
# Due to tensorboard log reading routine it's better to use rule:
# One writer (= monitor) --> one subdirectory.

# Essense metrics for one step:
step_monitor = BTgymMonitor(subdir='/Step/Metrics',
                            scalars={'step/reward',
                                     'step/drawdown',
                                     'step/broker_value',
                                    },
                           )
# ..for episode:
episode_monitor = BTgymMonitor(subdir='/Episode',
                               scalars={'process/episode/cpu_time_sec',
                                        'episode/value',
                                        'episode/reward'
                                       },
                              )
# State pictures:
images_monitor = BTgymMonitor(subdir='/Step/Image',
                              images={'human', 'agent', 'episode'},
                             )


# Process resources usage by related processes:
env_monitor = BTgymMonitor(subdir='/Process/BTgymEnvironment',
                           scalars={'process/cpu_usage_percent',
                                    'process/vmemory_usage_percent',
                                   },
                          )

server_monitor = BTgymMonitor(subdir='/Process/BTgymServer',
                              scalars={'process/cpu_usage_percent',
                                       'process/vmemory_usage_percent',
                                      },
                             )

tb_monitor = BTgymMonitor(subdir='/Process/TensorboardServer',
                          scalars={'process/cpu_usage_percent',
                                   'process/vmemory_usage_percent',
                                  },
                         )

system_monitor = BTgymMonitor(subdir='/Process/System',
                              scalars={'process/cpu_usage_percent',
                                       'process/vmemory_usage_percent',
                                      },
                             )

###############################################

# Number episodes to run:
num_episodes = 500

# Retrieve step visualisations every... step
render_step = 100

# System recources usage to monitor:
# Environment (current process):
env_p = psutil.Process()

# BTgymServer process:
o = env.reset()
server_p = psutil.Process(env.server.pid)

# Tensorboard process:
# Here is the convinient method to start tensorboard server. Since we use all defaults,
# point webbrowser to localhost:6006 to see output. 
# Call it just once for any monitor.
system_monitor.tensorboard.start()
# ...procsess is here:
tb_p = system_monitor.tensorboard.process

# Main loop:
global_step = 0
for episode in range(num_episodes):
    o = env.reset()
    # For every-skip-frame do:
    for step in itertools.count():
        
        # Choose random action:
        rnd_action = int((env.action_space.n)*random.random())
        
        # Take a step:
        o, r, d, i = env.step(rnd_action)
        last_i = i[0] # pull most recent info record

        # Write step summary:
        step_monitor.write(
            feed_dict={'step/reward': r,
                       'step/drawdown': last_i['drawdown'],
                       'step/broker_value': last_i['broker_value'],
                      },
            global_step=global_step + last_i['step'],
            )

        # procesess:
        system_monitor.write(
            feed_dict={'process/cpu_usage_percent': psutil.cpu_percent(interval=None),
                       'process/vmemory_usage_percent': psutil.virtual_memory()[2],
                      },
            global_step=global_step + last_i['step'],
            )
        
        env_monitor.write(
            feed_dict={'process/cpu_usage_percent': env_p.cpu_percent(),
                       'process/vmemory_usage_percent': env_p.memory_percent(memtype="vms"),
                        },
            global_step=global_step + last_i['step'],
            )
        
        server_monitor.write(
            feed_dict={'process/cpu_usage_percent': server_p.cpu_percent(),
                       'process/vmemory_usage_percent': server_p.memory_percent(memtype="vms"),
                      },
            global_step=global_step + last_i['step'],
            )
        
        tb_monitor.write(
            feed_dict={'process/cpu_usage_percent': tb_p.cpu_percent(),
                       'process/vmemory_usage_percent': tb_p.memory_percent(memtype="vms"),
                      },
            global_step=global_step + last_i['step'],
            )
        
        # Heavy matplotlib memory leak test:
        if last_i['step'] % render_step == 0 or d:
            images_monitor.write(
                feed_dict={'human': env.render('human')[None,:],
                           'agent': env.render('agent')[None,:],
                           'episode': env.render('episode')[None,:],
                          },
                global_step=global_step + last_i['step'],
            )

        if d:
            break

    global_step += last_i['step']
    
    # Write after every episode:
    episode_stat = env.get_stat()
    episode_monitor.write(
        feed_dict={'process/episode/cpu_time_sec': episode_stat['runtime'].total_seconds(),
                   'episode/value': last_i['broker_value'],
                   'episode/reward': r,
                  },
        global_step=episode_stat['episode'],
    )
 
    print('Episode {}: {} actions made, global step: {}\n'.format(episode, step + 1, global_step))
       

In [ ]:
# Clean up:

env.close()

#This will shut down tensorboard:
system_monitor.tensorboard.stop()

# Close all monitor writers:
step_monitor.close()
episode_monitor.close()
images_monitor.close()
system_monitor.close()
env_monitor.close()
server_monitor.close()
tb_monitor.close()


In [ ]:
env.renderer.params